In [8]:
import os
import openai
import pandas as pd
from dotenv import load_dotenv

config = dotenv.dotenv_values(".env")

openai.api_key = config['OPENAI_API_KEY']
openai.api_key = config['OPENAI_API_KEY']


openai.api_key_path = '.venv/.env'

def generate_athena_query(natural_language_query):
    # Preprocess and format the query as necessary
    # Make an API call to the GPT-3.5 model to generate the Athena query
    response = openai.Completion.create(
        engine='text-davinci-003',
        prompt=natural_language_query,
        max_tokens=100,
        temperature=0,
        n=1,
        stop=None
    )
    athena_query = response.choices[0].text.strip()
    return athena_query

df = pd.read_json("/Users/rohitmishra/Desktop/lom_athena_schema.json")
query = "How many offers do not have any applications in the past 2 days"
response = generate_athena_query(query)

ValueError: Malformed API key in .venv/.env.